In [1]:
%load_ext autoreload
%autoreload 2

In [20]:
import os
os.environ["TORCH_HOME"] = '/disks/sdb/torch_home'


%load_ext autoreload
%autoreload 2
%load_ext snoop


import itertools
import pandas
import csv
import torch
import numpy as np
from sklearn.manifold import TSNE
from matplotlib import pyplot as plt
# from allennlp.data.token_indexers.bert_tokenizers import PretrainedBertIndexer

from awesome_glue.task import Task
from awesome_glue.utils import AttackMetric
from awesome_glue.config import Config

from luna import ram_write, chunks
from allennlpx import allenutil

from luna import ram_read

from allennlpx.interpret.attackers.embedding_searcher import EmbeddingSearcher

from collections import Counter, defaultdict
from tqdm import tqdm

from copy import deepcopy
from typing import List

# pylint: disable=protected-access
import random
from collections import defaultdict
from functools import lru_cache
from itertools import product
from typing import List

import numpy as np
import torch
from allennlp.common.util import JsonDict, sanitize
from allennlp.data.fields import TextField
from allennlp.data.token_indexers import (ELMoTokenCharactersIndexer,
                                          TokenCharactersIndexer)
from allennlp.data.tokenizers import SpacyTokenizer, Token
from allennlp.modules.text_field_embedders.text_field_embedder import \
    TextFieldEmbedder
from allennlp.modules.token_embedders import Embedding

from allennlpx import allenutil
from allennlpx.interpret.attackers.attacker import (DEFAULT_IGNORE_TOKENS,
                                                    Attacker)
from allennlpx.interpret.attackers.embedding_searcher import EmbeddingSearcher
from allennlpx.interpret.attackers.policies import (CandidatePolicy,
                                                    EmbeddingPolicy,
                                                    SynonymPolicy)
from allennlpx.interpret.attackers.synonym_searcher import SynonymSearcher
from luna import cast_list, lazy_property, time_record
import copy

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


I0319 16:24:00.207964 139878374930240 file_utils.py:38] PyTorch version 1.4.0 available.
I0319 16:24:01.533295 139878374930240 modeling.py:230] Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .
Loading faiss with AVX2 support.
I0319 16:24:01.706631 139878374930240 modeling_bert.py:244] Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .


In [2]:
config = Config()
# config.finetunable=True
task = Task(config) # type: Task
task.from_pretrained()
config

cache for SST-spacy exists
*** load SST-spacy from cache cost 2.75 seconds
cache for 2f2a46e51336b27a7262689b00016955 exists
*** load 2f2a46e51336b27a7262689b00016955 from cache cost 0.0351 seconds
Restore model from checkpoint saved/models/SST-lstm-glove@0.ckpt.


Basic Args:
	--task_id=SST
	--finetunable=True
	--arch=lstm
	--pretrain=glove
	--_model_name=SST-lstm-glove
	--mode=attack
	--adv_data=nogit/SST-bert.adv.tsv
	--aug_data=
	--attack_vectors=glove
	--attack_data_split=dev
	--attack_size=100
	--attack_gen_aug=False
	--attack_gen_adv=False
	--alchemist=False
	--seed=2
Deduced Args:
	--model_name=SST-lstm-glove
	--tokenizer=spacy

In [15]:
from luna.registry import setup_registry
register, R = setup_registry('transforms')

In [16]:
@register('a')
class A:
    pass

In [18]:
R['a']()

In [24]:
import inspect
sig = inspect.signature(Task.__init__)


In [35]:
# import fastnumbers
fastnumbers.fast_real('')

''